# seq2seq模型——机器翻译

### 案例简介

seq2seq是神经机器翻译的主流框架，如今的商用机器翻译系统大多都基于其构建，在本案例中，我们将使用由NIST提供的中英文本数据训练一个简单的中英翻译系统，在实践中学习seq2seq的具体细节，以及了解机器翻译的基本技术。

### seq2seq模型

从根本上讲，机器翻译需要将输入序列（源语言中的单词）映射到输出序列（目标语言中的单词）。正如我们在课堂上讨论的那样，递归神经网络（RNN）可有效处理此类顺序数据。机器翻译中的一个重要难题是输入和输出序列之间没有一对一的对应关系。即，序列通常具有不同的长度，并且单词对应可以是不平凡的（例如，彼此直接翻译的单词可能不会以相同的顺序出现）。

为了解决这个问题，我们将使用一种更灵活的架构，称为seq2seq模型。该模型由编码器和解码器两部分组成，它们都是RNN。编码器将源语言中的单词序列作为输入，并输出RNN层的最终隐藏状态。解码器与之类似，除了它还具有一个附加的全连接层（带有softmax激活），用于定义翻译中下一个单词的概率分布。以此方式，解码器本质上用作目标语言的神经语言模型。关键区别在于，解码器将编码器的输出用作其初始隐藏状态，而不是零向量。

### 数据和代码

本案例使用了一个小规模的中英平行语料数据，并提供了一个简单的seq2seq模型实现，包括数据的预处理、模型的训练、以及简单的评测。

### 评分要求

分数由两部分组成，各占50%。第一部分得分为对于简单seq2seq模型的改进，并撰写实验报告，改进方式多样，下一小节会给出一些可能的改进方向。第二分部得分为测试数据的评测结果，我们将给出一个中文测试数据集（`test.txt`），其中每一行为一句中文文本，需要同学提交模型做出的对应翻译结果，助教将对于大家的提交结果统一机器评测，并给出分数。

### 改进方向

初级改进：
- 将RNN模型替换成GRU或者LSTM
- 使用双向的encoder获得更好的源语言表示
- 对于现有超参数进行调优，这里建议划分出一个开发集，在开发集上进行grid search，并且在报告中汇报开发集结果
- 引入更多的训练语料（如果尝试复杂模型，更多的训练数据将非常关键）

进阶改进：
- 使用注意力机制（注意力机制是一个很重要的NMT技术，建议大家优先进行这方面的尝试，具体有许多种变体，可以参考这个[综述](https://nlp.stanford.edu/pubs/emnlp15_attn.pdf)）
- 在Encoder部分，使用了字级别的中文输入，可以考虑加入分词的结果，并且将Encoder的词向量替换为预训练过的词向量，获得更好的性能

复杂改进：
- 使用beam search的技术来帮助更好的解码，对于beam-width进行调优
- 将RNN替换为Transformer模型，以及最新的改进变体

## 环境依赖

In [1]:
import unicodedata
import string
import re
import random
import time
import math

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

#import hanlp

from nltk.translate.bleu_score import sentence_bleu
import zhconv

如果使用GPU，则将下面的变量设为`True`。

In [2]:
USE_CUDA = True
print(USE_CUDA)

True


In [3]:
HanLP = hanlp.load(hanlp.pretrained.mtl.CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_SMALL_ZH)
result = HanLP('商品和服务', tasks = 'tok')
print(result)

{
  "tok/fine": [
    "商品",
    "和",
    "服务"
  ]
}


In [3]:
filename = 'translation2019zh_valid.json'
with open(filename, 'r', encoding = 'utf8') as f:
    string = f.read()

data = string.split('\n')
data.pop()

''

## 读取数据

我们将读取目录下的`cn-eng.txt`文件，其中每一行是一个平行句对，例子如下：

```
我很冷。    I am cold.
```

### 对于单词进行编号

这里引入了两个特殊符号，“SOS”即“Start of sentence”和“EOS”即“End of sentence”。他们会加到输入文本的两端，以控制解码过程。

In [23]:
PAD_INDEX = 0
UNK_INDEX = 1
SOS_INDEX = 2
EOS_INDEX = 3

# Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalize_string(s):
    s = unicode_to_ascii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z\u4e00-\u9fa5.!?，。？]+", r" ", s)
    return s

def read_sentence_file(filename):
    src_sentences_list = []
    trg_sentences_list = []
    src_vocab = []
    trg_vocab = []
    num = 0
    with open(filename, "r", encoding = 'utf8') as f:
        for line in f:
            num += 1
            if num%5000 == 0:
                print(num)
            src, trg = [normalize_string(s) for s in line.split('\t')]
            #src = zhconv.convert(src, 'zh-hans')
            #print(trg)
            temp = []
#             temp = HanLP(src, tasks = 'tok')['tok/fine']
            for w in src:
                if w != ' ':
                    temp.append(w)
                if w not in src_vocab:
                    src_vocab.append(w)
                    
            for w in temp:
                if w != ' ' and w not in src_vocab:
                    src_vocab.append(w)
            src_sentences_list.append(temp)
            
            temp = []
            for w in trg.split(' '):
                temp.append(w)
                if w not in trg_vocab:
                    trg_vocab.append(w)
            trg_sentences_list.append(temp)
            
        new_src_sentences_list, new_trg_sentences_list = [], []
        num = 0
        for src_sent, trg_sent in zip(src_sentences_list, trg_sentences_list):
            num += 1
            if num%5000 == 0:
                print(num)
            if len(src_sent) <= MAX_LENGTH and len(trg_sent) <= MAX_LENGTH:
                new_src_sentences_list.append(src_sent)
                new_trg_sentences_list.append(trg_sent)
                    
    return new_src_sentences_list, new_trg_sentences_list, src_vocab, trg_vocab

MAX_LENGTH = 20
MAX_SENT_LENGTH_PLUS_SOS_EOS = 50
src_sentences_list, trg_sentences_list, src_vocab, trg_vocab = read_sentence_file('cn-eng.txt')
print(len(src_sentences_list))
# src_extra_list = []
# trg_extra_list = []
# for dict_pair in data:
#     dict_pair = eval(dict_pair)
#     src = normalize_string(dict_pair['chinese'])
#     src_list = []
#     for w in src:
#         if w != ' ':
#             src_list.append(w)
#     if len(src_list) > MAX_LENGTH:
#     trg = normalize_string(dict_pair['english'])
#     #src_extra_list.append([w for w in src])
#     trg_extra_list.append([w for w in trg.split(' ')])

# src_sentences_list = src_sentences_list + src_extra_list
# trg_sentences_list = trg_sentences_list + trg_extra_list

train_src_sentences_list = src_sentences_list[:len(src_sentences_list)//100*95]
train_trg_sentences_list = trg_sentences_list[:len(trg_sentences_list)//100*95]
val_src_sentences_list = src_sentences_list[len(src_sentences_list)//100*95:]
val_trg_sentences_list = trg_sentences_list[len(trg_sentences_list)//100*95:]
for src_sent, trg_sent in zip(val_src_sentences_list, val_trg_sentences_list):
    if src_sent in train_src_sentences_list or trg_sent in train_trg_sentences_list:
        val_src_sentences_list.remove(src_sent)
        val_trg_sentences_list.remove(trg_sent)

length = [len(sent) for sent in train_src_sentences_list]

print(train_src_sentences_list[-1])
print(len(val_src_sentences_list))

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
85581
['汤', '姆', '求', '我', '让', '他', '早', '点', '回', '家', '。']
2743


### 文本预处理

丢弃除了中文、字母和常用标点之外的符号。

读取平行语料，并进行清理。

### 过滤句子

样例为了加快训练，只保留了不长于10个单词的句对，真正实验中将更多数据考虑进来可能获得更好的效果。

处理数据的全过程：

- 读取数据，每一行分别处理，将其转换成句对
- 对于文本进行处理，过滤无用符号
- 根据已有文本对于单词进行编号，构建符号到编号的映射


In [31]:
from torch.utils import data
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
device = "cuda" if torch.cuda.is_available() else "cpu"
import numpy as np
assert device == "cuda"   # use gpu whenever you can!

seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

class MTDataset(data.Dataset):
    def __init__(self, src_sentences, src_vocab, trg_sentences, trg_vocab, sampling=1.):
        self.src_sentences = src_sentences[:int(len(src_sentences) * sampling)]
        self.trg_sentences = trg_sentences[:int(len(src_sentences) * sampling)]

        self.max_src_seq_length = MAX_LENGTH+2
        self.max_trg_seq_length = MAX_LENGTH+2

        self.src_v2id = {v : i for i, v in enumerate(src_vocab)}
        self.src_id2v = {val : key for key, val in self.src_v2id.items()}
        self.trg_v2id = {v : i for i, v in enumerate(trg_vocab)}
        self.trg_id2v = {val : key for key, val in self.trg_v2id.items()}

    def __len__(self):
        return len(self.src_sentences)

    def __getitem__(self, index):
        src_sent = self.src_sentences[index]
        src_len = len(src_sent) + 2   # add <s> and </s> to each sentence
        src_id = []
        for w in src_sent:
            if w not in self.src_v2id:
                w = '<unk>'
            src_id.append(self.src_v2id[w])
        src_id = ([SOS_INDEX] + src_id + [EOS_INDEX] + [PAD_INDEX] *
                  (self.max_src_seq_length - src_len))

        trg_sent = self.trg_sentences[index]
        trg_len = len(trg_sent) + 2
        trg_id = []
        for w in trg_sent:
            if w not in self.trg_v2id:
                w = '<unk>'
            trg_id.append(self.trg_v2id[w])
        trg_id = ([SOS_INDEX] + trg_id + [EOS_INDEX] + [PAD_INDEX] *
                  (self.max_trg_seq_length - trg_len))

        return torch.tensor(src_id), src_len, torch.tensor(trg_id), trg_len

## 将文本数据转换为张量

为了训练，我们需要将句子变成神经网络可以理解的东西（数字）。每个句子将被分解成单词，然后变成张量，其中每个单词都被索引替换（来自之前的Lang索引）。在创建这些张量时，我们还将附加EOS令牌以表示该句子已结束。

![](https://i.imgur.com/LzocpGH.png)

In [32]:
# Return a list of indexes, one for each word in the sentence
def indexes_from_sentence(lang, sentence):
    if lang.name == 'cn':
        #sentence_list = HanLP(sentence, tasks = 'tok')['tok/fine'] 
        return [lang.word2index[word] for word in sentence]
    else:
        return [lang.word2index[word] for word in sentence.split(' ')]
    
def variable_from_sentence(lang, sentence):
    indexes = indexes_from_sentence(lang, sentence)
    indexes.append(EOS_token)
    var = torch.LongTensor(indexes).view(-1, 1)
    if USE_CUDA: var = var.cuda()
    return var

def variables_from_pair(pair):
    input_variable = variable_from_sentence(input_lang, pair[0])
    target_variable = variable_from_sentence(output_lang, pair[1])
    return (input_variable, target_variable)

# 组件模型

## 编码器

In [33]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1):
        super(EncoderRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, n_layers)
        
    def forward(self, word_inputs, hidden):
        # Note: we run this all at once (over the whole input sequence)
        seq_len = len(word_inputs)
        embedded = self.embedding(word_inputs).view(seq_len, 1, -1)
        output, hidden = self.rnn(embedded, hidden)
        return output, hidden

    def init_hidden(self):
        hidden = torch.zeros(self.n_layers, 1, self.hidden_size)
        if USE_CUDA: hidden = hidden.cuda()
        return hidden

In [41]:
class EncoderLSTM(nn.Module):
    # A torch module implementing an LSTM. The `forward` function should just
    # perform one step of update and output logits before softmax.

    def __init__(self, input_size, hidden_size, num_layers=2, bidirection = True, dropout=0.5):
        super(EncoderLSTM, self).__init__()
        
        self.input_size = input_size
        #print(input_size)
        #self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirection = bidirection
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers, batch_first=True,
                      dropout=dropout, bidirectional=self.bidirection)
        
        self.bi_linear = nn.Linear(2*hidden_size, hidden_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs, lengths):
        pack_pad = pack_padded_sequence(self.dropout(inputs), lengths.cpu(), batch_first=True, enforce_sorted=False)
        outputs, finals = self.gru(pack_pad)
        outputs, _ = pad_packed_sequence(outputs, batch_first=True, total_length = inputs.size(1))
        
        if self.bidirection:
            forward = finals[0:list(finals.size())[0]:2]
            backward = finals[1:list(finals.size())[0]:2]
            finals = torch.cat([forward, backward], dim=2)
            #print('final after')
            #print(finals.size())
            finals = self.bi_linear(finals)
            outputs = self.bi_linear(outputs)
        
        return outputs, finals

    def init_hidden(self):
        # Initialize hidden and memory states.
        if self.bidirection == True:
            return torch.zeros(self.num_layers*2, 1, self.hidden_size)
        else:
            return torch.zeros(self.num_layers, 1, self.hidden_size)

## 解码器

In [42]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1, dropout_p=0.1):
        super(DecoderRNN, self).__init__()
        
        # Keep parameters for reference
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        
        # Define layers
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, n_layers, dropout=dropout_p)
        self.out = nn.Linear(hidden_size, output_size)
    
    def forward(self, word_input, last_hidden):
        # Note: we run this one step at a time        
        word_embedded = self.embedding(word_input).view(1, 1, -1) # S=1 x B x N
        rnn_output, hidden = self.rnn(word_embedded, last_hidden)

        rnn_output = rnn_output.squeeze(0)
        output = F.log_softmax(self.out(rnn_output))

        return output, hidden

In [60]:
class BahAttention(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        
        self.hidden_size = hidden_size
        self.q_weight = nn.Linear(hidden_size, hidden_size, bias=False)
        self.k_weight = nn.Linear(hidden_size, hidden_size, bias=False)
        self.va = nn.Linear(hidden_size, 1, bias=False)
        self.att_weight = None
        
    def forward(self, q, k, v, mask):
        #q = q.unsqueeze(1)
        #q_linear = nn.Linear(q.size(2), self.hidden_size).cuda()
        #k_linear = nn.Linear(k.size(2), self.hidden_size).cuda()
        #print(list(q_p.size()))
        #k_p = self.k_weight(k)
        #print(list(k.size()))
        combine = torch.tanh(self.q_weight(q) + self.k_weight(k))
        energy = self.va(combine).squeeze(2).unsqueeze(1)
        energy.data.masked_fill_(mask == False, -float('inf'))
        att_weight = F.softmax(energy, dim=-1)
        self.att_weight = att_weight
        context = att_weight.bmm(v)
        return context

'''
AttentionDecoder
'''

class AttentionDecoder(nn.Module):
    """An attention-based RNN decoder."""

    def __init__(self, input_size, hidden_size, attention, num_layers, feed_ratio = 0.5, dropout=0.):
        """
          Inputs:
        - `input_size`, `hidden_size`, and `dropout` the same as in Encoder.
        - `attention`: this is your self-defined Attention object. You can
            either define an individual class for your Attention and pass it
            here or leave `attention` as None and just implement everything
            here.
        """
        super(AttentionDecoder, self).__init__()

        ### Your code here!
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.dropout = dropout
        self.feed_ratio = feed_ratio

        self.attention = attention
        self.input_hidden = nn.Linear(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size*2, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.drop_layer = nn.Dropout(dropout)
        self.output1 = nn.Linear(hidden_size, hidden_size)
        self.output2 = nn.Linear(hidden_size*3, hidden_size)

    
    def forward(self, inputs, encoder_hiddens, encoder_finals,  src_mask,
              hidden=None, trg_mask = None, max_len=None):
        """Unroll the decoder one step at a time.

        Inputs:
          - `inputs`: a 3d-tensor of shape (batch_size, max_seq_length, embed_size)
              representing a batch of padded embedded word vectors of target
              sentences (for teacher-forcing during training).
          - `encoder_hiddens`: a 3d-tensor of shape
              (batch_size, max_seq_length, hidden_size) representing the encoder
              outputs for each decoding step to attend to. 
          - `encoder_finals`: a 3d-tensor of shape
              (num_enc_layers, batch_size, hidden_size) representing the final
              encoder hidden states used to initialize the initial decoder hidden
              states.
          - `src_mask`: a 3d-tensor of shape (batch_size, 1, max_seq_length)
              representing the mask for source sentences.
          - `trg_mask`: a 3d-tensor of shape (batch_size, 1, max_seq_length)
              representing the mask for target sentences.
          - `hidden`: a 3d-tensor of shape (1, batch_size, hidden_size) representing
              the value to be used to initialize the initial decoder hidden states.
              If None, then use `encoder_finals`.
          - `max_len`: an int representing the maximum decoding length.

        Returns:
          - `outputs`: (same as in Decoder) a 3d-tensor of shape
              (batch_size, max_seq_length, hidden_size) representing the raw
              decoder outputs (before converting to a `trg_vocab_size`-dim vector).
          - `hidden`: a 3d-tensor of shape (1, batch_size, hidden_size)
              representing the last decoder hidden state.
        """

        # The maximum number of steps to unroll the RNN.
        if max_len is None:
            max_len = inputs.size(1)

        if hidden is None:
          #print('encoder final')
          #print(encoder_finals.size())
            hidden = self.init_hidden(encoder_finals)

        #outputs = None
        ### Your code here!

        #print('input')
        #print(inputs.size())
        #print(self.input_size)
        #print('max_len: ' + str(max_len))

        outputs = []
        for i in range(max_len):
            #print(hidden[-1].size())
            #print(hidden[-1].unsqueeze(1).size())
            context = self.attention(hidden[-1].unsqueeze(1), encoder_hiddens, encoder_hiddens, src_mask)
            if i == 0:
                embed_input = self.input_hidden(inputs[:,i,:].unsqueeze(1))
                lstm_input = torch.cat([context, self.drop_layer(embed_input)], dim = -1)
                #print('hidden size')
                #print(hidden[0].size())
                output, (hidden, cell) = self.lstm(lstm_input, (hidden, hidden))

            else:
                feed_target = True if random.random() < self.feed_ratio else False
                if feed_target:
                    embed_input = self.input_hidden(inputs[:,i,:].unsqueeze(1))
                else:
                    embed_input = output

                lstm_input = torch.cat([context, self.drop_layer(embed_input)], dim = -1)
                #print('hidden size')
                #print(hidden.size())
                output, (hidden, cell) = self.lstm(lstm_input, (hidden, hidden))


            result1 = self.output1(output + context)
            #result2 = self.output2(torch.cat([output, context, embed_input]), dim = -1)
            outputs.append(result1)
            #outputs.append(result2)
        
        #print(outputs[0].size())
        outputs = torch.cat(outputs, dim = 1)
        #print(outputs.size())
        
        return hidden, outputs

    def init_hidden(self, encoder_finals):
        """Use encoder final hidden state to initialize decoder's first hidden
        state."""
        #decoder_init_hiddens = None
        ### Your code here!

        if encoder_finals != None:
            return torch.tanh(encoder_finals)
        else:
            return None

In [61]:
class EncoderAttentionDecoder(nn.Module):
    """A Encoder-Decoder architecture with attention.
    """
    def __init__(self, encoder, decoder, src_embed , trg_embed, generator):
        """
        Inputs:
          - `encoder`: an `Encoder` object.
          - `decoder`: an `AttentionDecoder` object.
          - `src_embed`: an nn.Embedding object representing the lookup table for
              input (source) sentences.
          - `trg_embed`: an nn.Embedding object representing the lookup table for
              output (target) sentences.
          - `generator`: a `Generator` object. Essentially a linear mapping. See
              the next code cell.
        """
        super(EncoderAttentionDecoder, self).__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.trg_embed = trg_embed
        self.generator = generator

    def forward(self, src_ids, trg_ids, src_lengths):
        """Take in and process masked source and tar get sequences.

        Inputs:
          `src_ids`: a 2d-tensor of shape (batch_size, max_seq_length) representing
            a batch of source sentences of word ids.
          `trg_ids`: a 2d-tensor of shape (batch_size, max_seq_length) representing
            a batch of target sentences of word ids.
          `src_lengths`: a 1d-tensor of shape (batch_size,) representing the
            sequence length of `src_ids`.

        Returns the decoder outputs, see the above cell.
        """
        ### Your code here!
        # You can refer to `EncoderDecoder` and extend from it.
        src_mask = (src_ids != 0).unsqueeze(1)
        #print(src_ids.size())
        #print(src_mask.size())
        encoder_output, encoder_finals = self.encode(src_ids, src_lengths)
        #print('output')
        #print(encoder_output.size())
        return self.decode(encoder_finals, encoder_output, trg_ids[:, :-1], src_mask)
    
    def encode(self, src_ids, src_lengths):
        return self.encoder(self.src_embed(src_ids), src_lengths)
    
    def decode(self, encoder_finals, encoder_hiddens, trg_ids, src_mask, decoder_hidden=None):
        #print(trg_ids.size())
        #print(src_mask.size())
        return self.decoder(self.trg_embed(trg_ids), encoder_hiddens, encoder_finals, src_mask, decoder_hidden)

In [62]:
batch_size = 64
embed_size = 256   # Each word will be represented as a `embed_size`-dim vector.
hidden_size = 256  # RNN hidden size.
dropout = 0.2
num_layers = 2
bidirection = True
feed_ratio = 0.5
learning_rate = 5e-4
clipping_value = 1 
num_epochs = 15
MAX_LENGTH = 20


train_set = MTDataset(train_src_sentences_list, src_vocab,
                      train_trg_sentences_list, trg_vocab, sampling=1.)
train_data_loader = data.DataLoader(train_set, batch_size=batch_size,
                                    num_workers=8, shuffle=True)

val_set = MTDataset(val_src_sentences_list, src_vocab,
                    val_trg_sentences_list, trg_vocab, sampling=1.)
val_data_loader = data.DataLoader(val_set, batch_size=batch_size, num_workers=8, shuffle=False)

# 训练

## 一次训练迭代

为了训练，我们首先通过编码器逐字运行输入语句，并跟踪每个输出和最新的隐藏状态。接下来，为解码器提供解码器的最后一个隐藏状态作为其第一隐藏状态，并向其提供`<SOS>`作为其第一输入。从那里开始，我们迭代地预测来自解码器的下一个单词。
    
### Teacher Forcing 和 Scheduled Sampling

"Teacher Forcing"指的是每次都基于完全准确的上文进行解码，这样训练模型收敛很快，但是会造成实际场景和训练场景有较大差别，因为实际场景上文也都是模型预测的，可能不准确，具体细节可参考[论文](http://minds.jacobs-university.de/sites/default/files/uploads/papers/ESNTutorialRev.pdf)。

观察Teacher Forcing的网络的输出，我们可以看到该网络语法连贯，但是偏离正确的翻译。可以将其为学会了如何听老师的指示，而未学习如何独自冒险。

解决强迫教师问题的方法称为“计划抽样”（[Scheduled Sampling](https://arxiv.org/abs/1506.03099)），它在训练时仅在使用目标值和预测值之间进行切换。我们将在训练时随机选择,有时我们将使用真实目标作为输入（忽略解码器的输出），有时我们将使用解码器的输出。

In [63]:
import math


class SimpleLossCompute:
    """A simple loss compute and train function."""

    def __init__(self, generator, criterion, opt=None, c_value = None, model = None):
        self.generator = generator
        self.criterion = criterion
        self.opt = opt
        self.clipping_value = c_value
        self.model = model

    def __call__(self, x, y, norm):
        x = self.generator(x)
        loss = self.criterion(x.contiguous().view(-1, x.size(-1)),
                              y.contiguous().view(-1))
        loss = loss / norm

        if self.opt is not None:  # training mode
            loss.backward()    
            ########################## clip
            torch.nn.utils.clip_grad_norm(self.model.parameters(), self.clipping_value)
            ##########################
            self.opt.step()
            self.opt.zero_grad()

        return loss.data.item() * norm


def run_epoch(data_loader, model, loss_compute, print_every):
    """Standard Training and Logging Function"""

    total_tokens = 0
    total_loss = 0

    for i, (src_ids_BxT, src_lengths_B, trg_ids_BxL, trg_lengths_B) in enumerate(data_loader):
        # We define some notations here to help you understand the loaded tensor
        # shapes:
        #   `B`: batch size
        #   `T`: max sequence length of source sentences
        #   `L`: max sequence length of target sentences; due to our preprocessing
        #        in the beginning, `L` == `T` == 50
        # An example of `src_ids_BxT` (when B = 2):
        #   [[2, 4, 6, 7, ..., 4, 3, 0, 0, 0],
        #    [2, 8, 6, 5, ..., 9, 5, 4, 3, 0]]
        # The corresponding `src_lengths_B` would be [47, 49].
        # Note that SOS_INDEX == 2, EOS_INDEX == 3, and PAD_INDEX = 0.

        src_ids_BxT = src_ids_BxT.to(device)
        src_lengths_B = src_lengths_B.to(device)
        trg_ids_BxL = trg_ids_BxL.to(device)
        del trg_lengths_B   # unused

        _, output = model(src_ids_BxT, trg_ids_BxL, src_lengths_B)
        #print(output.size())
        loss = loss_compute(x=output, y=trg_ids_BxL[:, 1:],
                            norm=src_ids_BxT.size(0))
        total_loss += loss
        total_tokens += (trg_ids_BxL[:, 1:] != PAD_INDEX).data.sum().item()

        if model.training and i % print_every == 0:
            print("Epoch Step: %d Loss: %f" % (i, loss / src_ids_BxT.size(0)))

    return math.exp(total_loss / float(total_tokens))


def train(model, num_epochs, learning_rate, print_every, clipping_value):
    # Set `ignore_index` as PAD_INDEX so that pad tokens won't be included when
    # computing the loss.
    criterion = nn.NLLLoss(reduction="sum", ignore_index=PAD_INDEX)
    optim = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Keep track of dev ppl for each epoch.
    dev_ppls = []

    for epoch in range(num_epochs):
        print("Epoch", epoch)

        model.train()
        train_ppl = run_epoch(data_loader=train_data_loader, model=model,
                              loss_compute=SimpleLossCompute(model.generator,
                                                             criterion, optim, clipping_value, model),
                              print_every=print_every)

        model.eval()
        with torch.no_grad():      
            dev_ppl = run_epoch(data_loader=val_data_loader, model=model, loss_compute=SimpleLossCompute(model.generator, criterion, None, None), print_every=print_every)
            print("Validation perplexity: %f" % dev_ppl)
            dev_ppls.append(dev_ppl)
        
    return dev_ppls

下面是用于辅助输出训练情况的函数

In [64]:
class Generator(nn.Module):
    """Define standard linear + softmax generation step."""
    def __init__(self, hidden_size, vocab_size):
        super(Generator, self).__init__()
        self.proj = nn.Linear(hidden_size, vocab_size, bias=False)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

attention = BahAttention(hidden_size)
attn_seq2seq = EncoderAttentionDecoder(
  encoder=EncoderLSTM(embed_size, hidden_size, num_layers, bidirection, dropout),
  decoder=AttentionDecoder(embed_size, hidden_size, attention, num_layers, feed_ratio, dropout),
  src_embed=nn.Embedding(len(src_vocab), embed_size),
  trg_embed=nn.Embedding(len(trg_vocab), embed_size),
  generator=Generator(hidden_size, len(trg_vocab))).to(device)

attn_dev_ppls = train(attn_seq2seq, num_epochs, learning_rate, 100, clipping_value)

def plot_perplexity(perplexities):
    """plot perplexities"""
    plt.title("Perplexity per Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Perplexity")
    plt.plot(perplexities)

plot_perplexity(attn_dev_ppls)

Epoch 0


/home/mingye/.conda/envs/opence/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Epoch Step: 0 Loss: 80.958023
Epoch Step: 100 Loss: 44.327747
Epoch Step: 200 Loss: 45.000881
Epoch Step: 300 Loss: 43.000919
Epoch Step: 400 Loss: 35.433319
Epoch Step: 500 Loss: 34.070183
Epoch Step: 600 Loss: 34.664841
Epoch Step: 700 Loss: 31.802898
Epoch Step: 800 Loss: 34.624905
Epoch Step: 900 Loss: 30.498161
Epoch Step: 1000 Loss: 30.000204
Epoch Step: 1100 Loss: 36.287941
Epoch Step: 1200 Loss: 28.091522
Validation perplexity: 49.292940
Epoch 1
Epoch Step: 0 Loss: 24.345860
Epoch Step: 100 Loss: 25.436298
Epoch Step: 200 Loss: 24.317717
Epoch Step: 300 Loss: 29.237633
Epoch Step: 400 Loss: 22.297001
Epoch Step: 500 Loss: 24.712944
Epoch Step: 600 Loss: 25.667643
Epoch Step: 700 Loss: 20.795748
Epoch Step: 800 Loss: 24.424608
Epoch Step: 900 Loss: 27.347464
Epoch Step: 1000 Loss: 23.321180
Epoch Step: 1100 Loss: 21.581831
Epoch Step: 1200 Loss: 22.232965
Validation perplexity: 40.156180
Epoch 2
Epoch Step: 0 Loss: 22.971785
Epoch Step: 100 Loss: 19.244396
Epoch Step: 200 Loss: 

NameError: name 'plt' is not defined

## 进行训练

以下设置变量用于绘制图标和跟踪进度：

要进行实际训练，我们会多次调用训练函数，并在进行过程中打印中间信息。

In [18]:
plt.title("Perplexity per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Perplexity")
plt.plot(pure_dev_ppls, label='pure model')
plt.plot(attn_dev_ppls, label='attn model')
plt.legend(loc = 'upper right')

def greedy_decode(model, src_ids, src_lengths, max_len, pad_index = 0):
    """Greedily decode a sentence for EncoderDecoder."""

    with torch.no_grad():
        encoder_hiddens, encoder_finals = model.encode(src_ids, src_lengths)
        prev_y = torch.ones(1, 1).fill_(SOS_INDEX).type_as(src_ids)

    output = []
    hidden = None
    attn_scores = []

    for i in range(max_len):
        with torch.no_grad():
            if model == pure_seq2seq:
                hidden, outputs = model.decode(encoder_finals, prev_y, hidden)
            if model == attn_seq2seq:
                src_mask = (src_ids != pad_index).unsqueeze(1)
                hidden, outputs = model.decode(encoder_finals, encoder_hiddens, prev_y, src_mask, hidden)
                attn_score = model.decoder.attention.att_weight.cpu().numpy()
                attn_scores.append(attn_score)
            
            prob = model.generator(outputs[:, -1])

        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.data.item()
        output.append(next_word)
        prev_y = torch.ones(1, 1).type_as(src_ids).fill_(next_word)

    output = np.array(output)
    attn_scores = np.array(attn_scores)

    # Cut off everything starting from </s>.
    first_eos = np.where(output == EOS_INDEX)[0]
    if len(first_eos) > 0:
        output = output[:first_eos[0]]

    #print(np.array(attn_scores))
    return output, attn_scores.reshape(max_len, -1)
  

def lookup_words(x, vocab):
    return [vocab[i] for i in x]

In [19]:
def print_examples(model, data_loader, n=5,
                   max_len=MAX_SENT_LENGTH_PLUS_SOS_EOS, 
                   src_vocab_set=src_vocab_set, trg_vocab_set=trg_vocab_set):
    """Prints `n` examples. Assumes batch size of 1."""

    model.eval()

    preds = []
    weights = []

    for i, (src_ids, src_lengths, trg_ids, _) in enumerate(data_loader):
        pred, attn_scores = greedy_decode(model, src_ids.to(device), src_lengths.to(device),
                               max_len=max_len)

        #preds.append(result)
        #weights.append(attn_scores)

        # remove <s>
        src_ids = src_ids[0, 1:]
        trg_ids = trg_ids[0, 1:]

        # remove </s> and <pad>
        src_ids = src_ids[:np.where(src_ids == EOS_INDEX)[0][0]]
        trg_ids = trg_ids[:np.where(trg_ids == EOS_INDEX)[0][0]]

        src_w = lookup_words(src_ids, src_vocab_set)
        #trg_w = lookup_words(trg_ids, trg_vocab_set)
        pred_w = lookup_words(pred, trg_vocab_set)

        print("Example #%d" % (i + 1))
        print("Src : ", " ".join(lookup_words(src_ids, vocab=src_vocab_set)))
        print("Trg : ", " ".join(lookup_words(trg_ids, vocab=trg_vocab_set)))
        print("Pred: ", " ".join(lookup_words(pred, vocab=trg_vocab_set)))
        #print()

        if i == n - 1:
            break
    
    return attn_scores, src_w, pred_w

In [20]:
import sacrebleu
from tqdm import tqdm


def compute_BLEU(model, data_loader):
    bleu_score = []

    model.eval()
    for src_ids, src_lengths, trg_ids, _ in tqdm(data_loader):
        result, attn_score = greedy_decode(model, src_ids.to(device), src_lengths.to(device),
                               max_len=MAX_SENT_LENGTH_PLUS_SOS_EOS)
        # remove <s>
        src_ids = src_ids[0, 1:]
        trg_ids = trg_ids[0, 1:]
        # remove </s> and <pad>
        src_ids = src_ids[:np.where(src_ids == EOS_INDEX)[0][0]]
        trg_ids = trg_ids[:np.where(trg_ids == EOS_INDEX)[0][0]]

        pred = " ".join(lookup_words(result, vocab=trg_vocab_set))
        targ = " ".join(lookup_words(trg_ids, vocab=trg_vocab_set))

        bleu_score.append(sacrebleu.raw_corpus_bleu([pred], [[targ]], .01).score)

    return bleu_score


# test_set = MTDataset(test_src_sentences_list, src_vocab,
#                      test_trg_sentences_list, trg_vocab, sampling=1.)
# test_data_loader = data.DataLoader(test_set, batch_size=1, num_workers=8,
#                                    shuffle=False)

print('BLEU score: %f' % (np.mean(compute_BLEU(attn_seq2seq,
                                               val_data_loader))))

'他不吃生鱼。'

In [22]:
def beam_evaluate(sentence, encoder, decoder, beam_width, max_length=MAX_LENGTH):
    input_variable = variable_from_sentence(input_lang, sentence)
    input_length = input_variable.size()[0]
    #target_variable = variable_from_sentence(input_lang, target)
    
    # Run through encoder
    #encoder_hidden = encoder.init_hidden()
    #encoder_outputs, encoder_hidden = encoder(input_variable, encoder_hidden)
    encoder_outputs, encoder_hidden = encoder(input_variable)
    # Create starting vectors for decoder
    decoder_input = torch.LongTensor([[SOS_token]]) # SOS
    if USE_CUDA:
        decoder_input = decoder_input.cuda()
    
    candidates = [(0, decoder_input, [])]
    decoder_hidden = encoder_hidden
    
    decoded_words = []
    #decoded_words_idx = []
    decoder_attentions = torch.zeros(max_length, max_length)
    
    #log_p = 0
    # Run through decoder
    final_candidates = []
    for di in range(max_length):
        #if len(final_candidates) == beam_width or beam_width <= 0:
            #break
            
        next_candidates = []
        for log_p, decoder_input, words in candidates:
            #decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)
            for idx in range(output_lang.n_words):
                w = output_lang.index2word[idx]
                if w == 'SOS' or 'UNK':
                    continue
                elif w == 'EOS':
                    count = len(pairs)
                    decoder_output[0, idx] = decoder_output[0, idx]*count/1000
                else:
                    count = output_lang.word2count[w]
                    if count > 5000:
                        decoder_output[0, idx] = decoder_output[0, idx]*count/1000

            # Choose top word from output
            topv, topi = decoder_output.data.topk(beam_width)
            topv = torch.add(topv, log_p)
            for i in range(beam_width):
                #print(topi[0][i])
                word_i = output_lang.index2word[topi[0][i].item()]
                #print(word_i)
                next_candidates.append([topv[0][i], topi[0][i], words + [word_i]])
        
        next_candidates.sort(reverse = True)
        next_candidates = next_candidates[:beam_width]
        candidates = []
        for i in range(len(next_candidates)):
            if EOS_token == next_candidates[i][1] or di == max_length-1:
                beam_width -= 1
                final_candidates.append(next_candidates[i])
            else:
                next_candidates[i][1] = torch.LongTensor([[next_candidates[i][1]]])
                if USE_CUDA: next_candidates[i][1] = next_candidates[i][1].cuda()
                candidates.append(next_candidates[i])
        
#         # Next input is chosen word
#         decoder_input = torch.LongTensor([[ni]])
#         if USE_CUDA: decoder_input = decoder_input.cuda()
    
    max_log_p = -float('inf')
    best_decode = []
    for candidate in final_candidates:
        print(candidate[2])
        candidate[0] = candidate[0].item()/len(candidate[2])
        if candidate[0] > max_log_p:
            max_log_p = candidate[0]
            best_decode = candidate[2]
        
    return best_decode

#words = evaluate(pairs[1][0], encoder, decoder)

## 绘制训练loss

# 模型验证

随机选取一个句子进行验证。

In [ ]:
beam_width = 5
def evaluate_randomly(encoder, decoder):
    pair = random.choice(pairs)
    
    #output_words = beam_evaluate(pair[0], encoder, decoder, beam_width)
    output_words = beam_evaluate(pair[0], encoder, decoder, beam_width)
    output_sentence = ' '.join(output_words)
    
    print('>', pair[0])
    print('=', pair[1])
    print('<', output_sentence)
    print('')

In [ ]:
for i in range(10):
    evaluate_randomly(encoder, decoder)

随机的验证只是一个简单的例子，为了能系统性的完成测试数据的翻译，这里仍需要实现一个新的函数。